In [34]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd

def load_data(ticker):

    filename = '../data/' + ticker + '_1min_firstratedata.csv'
    df = pd.read_csv(filename)
    df['ticker'] = ticker

    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["date"] = df["timestamp"].dt.date
    df["time"] = df["timestamp"].dt.time
    return df

ticker = 'SPY'
df = load_data(ticker)

In [35]:
# Add the RSI indicator with period n to a df

def RSI(df, n):
    "function to calculate RSI"
    delta = df["close"].diff()
    delta = delta[1:]
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    df["up"] = up.round(4)
    df["down"] = down.round(4)
    AVG_Gain = df["up"].rolling(window=n).mean()
    AVG_Loss = abs(df["down"].rolling(window=n).mean())
    RS = AVG_Gain / AVG_Loss
    RSI = 100.0 - (100.0 / (1.0 + RS))
    df["RSI_14"] = RSI.round(4)
    df = df.drop(columns=["up", "down"])

    return df

df = RSI(df, 14)

#df.head()

In [36]:
import datetime
import pandas as pd

def simulate_trade_RSI(date, entry, exit_high, exit_low, max_trades, df):
    df_day = df[(df["date"] == date) & (df["time"] >= datetime.time(9, 30, 0)) & (df["time"] <= datetime.time(16, 0, 0))]
    trade_log = pd.DataFrame(columns=["Date", "Direction", "Entry", "Exit", "TimeEnter", "TimeExit", "Gain%", "Gain$", "ExitCondition"])
    trades, in_trade = 0, False

    for i in range(1, len(df_day)):
        entry_rsi = (df_day["RSI_14"].iloc[i - 1] < entry and df_day["RSI_14"].iloc[i] >= entry)

        exit_profit = (df_day["close"].iloc[i - 1] >= exit_high and df_day["close"].iloc[i] < exit_high)
        exit_stoploss = (df_day["close"].iloc[i - 1] <= exit_low and df_day["close"].iloc[i] > exit_low)
        exit_endofday = (i == len(df_day) - 1)
        exit_condition = "MaxProfit" if exit_profit else "StopLoss" if exit_stoploss else "EndOfDay" if exit_endofday else None

        if not in_trade and entry_rsi:
            entry_price, in_trade, time_enter = df_day["close"].iloc[i], True, df_day["time"].iloc[i - 1]

        elif in_trade and (exit_profit or exit_stoploss or exit_endofday):
            exit_price = df_day["close"].iloc[i]
            trade_gain_percent = round(((exit_price - entry_price) / entry_price) * 100, 2)
            trade_gain_net = round(exit_price - entry_price, 2)

            trade_log.loc[len(trade_log)] = pd.Series({
                "Date": date,
                "Direction": "Long",
                "Entry": entry_price,
                "Exit": exit_price,
                "TimeEnter": time_enter,
                "TimeExit": df_day["time"].iloc[i],
                "Gain%": trade_gain_percent,
                "Gain$": trade_gain_net,
                "ExitCondition": exit_condition
            })

            trades, entry_price, in_trade = trades + 1, 0, False

            # Check if the maximum number of trades is reached
            if trades >= max_trades:
                break

    return trade_log


In [37]:
# Combine the simulated trades for a range of dates into a single dataframe

def simulate_date_range(start_date, end_date, entry, exit_high, exit_low, max_trades, df):
    day_datas = []

    for n in range((end_date - start_date).days + 1):
        date = start_date + datetime.timedelta(n)
        day_data = simulate_trade_RSI(date, entry, exit_high, exit_low, max_trades, df)
        if day_data is not None:
            day_datas.append(day_data)

    day_datas = pd.concat(day_datas, ignore_index=True)

    return day_datas

In [38]:
trades = simulate_date_range(datetime.date(2022, 9, 30), datetime.date(2023, 9, 30), 30, 70, 20, 2, df)

In [39]:
trades
# histogram of the trade gain%

import plotly.express as px
import plotly.graph_objects as go

fig = px.histogram(trades, x="Gain%", nbins=100, title="Histogram of Trade Gain%")
fig.update_layout(
    xaxis_title="Gain%",
    yaxis_title="Count",
    bargap=0.1
)
fig.show()

In [40]:
print("Total Trades:", len(trades))
print("Total Gain%:", round(trades["Gain%"].sum(), 2))
print("Average Gain%:", round(trades["Gain%"].mean(), 5))
print("Median Gain%:", round(trades["Gain%"].median(), 2))

print("Average Gain$:", round(trades["Gain$"].mean(), 5))
print("Median Gain$:", round(trades["Gain$"].median(), 2))


trades["Gain%"].describe()

Total Trades: 251
Total Gain%: 6.54
Average Gain%: 0.02606
Median Gain%: 0.03
Average Gain$: 0.08235
Median Gain$: 0.11


count     251.00
unique    162.00
top         0.29
freq        5.00
Name: Gain%, dtype: float64

In [51]:
trades.to_csv('../temp/trades.csv', index=False)

,Date,Direction,Entry,Exit,TimeEnter,TimeExit,Gain%,Gain$,ExitCondition
0,2022-09-30,Long,364.67,357.87,11:38:00,16:00:00,-1.86,-6.8,EndOfDay
1,2022-10-03,Long,360.2711,366.77,09:41:00,16:00:00,1.8,6.5,EndOfDay
2,2022-10-04,Long,376.389,377.74,10:26:00,16:00:00,0.36,1.35,EndOfDay
3,2022-10-05,Long,372.29,377.54,10:13:00,16:00:00,1.41,5.25,EndOfDay
4,2022-10-06,Long,374.6999,373.21,10:14:00,16:00:00,-0.4,-1.49,EndOfDay
5,2022-10-07,Long,368.53,362.78,09:33:00,16:00:00,-1.56,-5.75,EndOfDay
6,2022-10-10,Long,362.89,360.23,09:36:00,16:00:00,-0.73,-2.66,EndOfDay
7,2022-10-11,Long,358.3461,357.73,09:30:00,16:00:00,-0.17,-0.62,EndOfDay
8,2022-10-12,Long,357.97,356.79,09:34:00,16:00:00,-0.33,-1.18,EndOfDay
9,2022-10-13,Long,352.48,366.06,10:29:00,16:00:00,3.85,13.58,EndOfDay


In [41]:
fig = px.histogram(trades, x="TimeEnter", nbins=50, title="Histogram of Time to Enter Trade")
fig.update_layout(
    xaxis_title="Time",
    yaxis_title="Count",
    bargap=0.1
)
fig.show()

In [42]:
# scatter plot of entry time vs gain%

fig = px.scatter(trades, x="TimeEnter", y="Gain%", title="Entry Time vs Gain%")
fig.update_layout(
    xaxis_title="Time",
    yaxis_title="Gain%",
    bargap=0.1
)

fig.show()

In [43]:
def rsi_ranged_backtest_single_pnl(df, rsi_enter, rsi_exit, maxTrades, ticker):
    res = pd.DataFrame()
    res.loc[0, 'Ticker'] = ticker
    res.loc[0, 'StartDate'] = df['Date'].min()
    res.loc[0, 'EndDate'] = df['Date'].max()
    res.loc[0, 'Entry'] = rsi_enter
    res.loc[0, 'Exit'] = rsi_exit,
    res.loc[0, 'MaxTrades'] = maxTrades,
    res.loc[0, 'TotalTrades'] = len(df)
    res.loc[0, 'TotalPercentGain'] = round(df['Gain%'].sum(), 2)
    res.loc[0, 'AveragePercentGain'] = round(df['Gain%'].mean(), 2)
    res.loc[0, 'MaxPercentGain'] = round(df['Gain%'].max(), 2)
    res.loc[0, 'MinPercentGain'] = round(df['Gain%'].min(), 2)

    return res

In [44]:
def multi_rsi_backtest(tickers, max_trades, indicator_pairs):
    results = pd.DataFrame(
        columns=[
            "Ticker","StartDate","EndDate","Entry","Exit","TotalTrades",
            "TotalPercentGain","AveragePercentGain","MaxPercentGain","MinPercentGain"])

    for e in indicator_pairs:
        for f in max_trades:
            for g in tickers:

                df = load_data(g)
                df = RSI(df, 14)

                print("testing TICKER/ENTER/EXIT/MAXTRADES:", g, e[0], e[1], f)
                trades = simulate_date_range(datetime.date(2022, 9, 30), datetime.date(2023, 9, 30), e[0], e[1], f, df)
                pnl = rsi_ranged_backtest_single_pnl(trades, e[0], e[1], f, g)
                results.loc[len(results)] = pnl.loc[0]

    results = results.sort_values(by=["TotalPercentGain"], ascending=False)
    return results